In [2]:
import pandas as pd
import sys
sys.path.append ('..')
from src.models.model1.recommend import get_technique_tatic_stage, initialize_technique_interaction_rate
from src.data.build_features_3 import build_feature_interaction_frequency
label_df = pd.read_pickle ('scripts/tmp_m1pp_label.pkl')
technique_df = pd.read_pickle ('data/processed/model1/processed_technnique_features.pkl')
tactics_order_df = pd.read_csv ('data/raw/tactics_order.csv', index_col= 0)

In [4]:
res = initialize_technique_interaction_rate (label_df= label_df, technique_df= technique_df, mode = 'avg')

In [5]:
res

,technique_ID,input_technique_data_sources,input_technique_detection_name,input_technique_mitigation_id,input_technique_platforms,input_technique_software_id,input_technique_tactics,input_technique_description,input_technique_interaction_rate
0,T1548,"[command_command_execution, file_file_metadata...","[command_execution, file_metadata, file_modifi...","[m1047, m1038, m1028, m1026, m1022, other]","[linux, windows, macos]",[],"[defense_evasion, privilege_escalation]","[-0.36630338, -0.24036503, -0.36260876, 0.0984...",-0.174181
1,T1548.002,"[command_command_execution, process_process_cr...","[command_execution, process_creation, windows_...","[m1047, m1026, other]",[windows],"[s0584, s0640, s0606, s0570, s1068, s0089, s10...","[defense_evasion, privilege_escalation]","[-0.5305501, -0.43114784, -0.67361057, 0.16222...",0.477944
2,T1548.004,"[process_os_api_execution, process_process_cre...","[os_api_execution, process_creation]",[m1038],[macos],[s0402],"[defense_evasion, privilege_escalation]","[-0.57463825, -0.65824693, -0.23797776, -0.254...",-0.174181
3,T1548.001,"[command_command_execution, file_file_metadata...","[command_execution, file_metadata, file_modifi...",[m1028],"[linux, macos]",[other],"[defense_evasion, privilege_escalation]","[-0.7420097, -0.461457, -0.45979652, -0.116038...",-0.174181
4,T1548.003,"[command_command_execution, file_file_modifica...","[command_execution, file_modification, process...","[m1028, m1026, m1022]","[linux, macos]","[s0154, s0279, other]","[defense_evasion, privilege_escalation]","[-0.8037999, -0.6815256, 0.18332177, 0.0635958...",-0.174181
...,...,...,...,...,...,...,...,...,...
602,T1102.002,"[network_traffic_network_connection_creation, ...","[network_connection_creation, network_traffic_...","[m1031, other]","[linux, windows, macos]","[s0128, s0657, s0660, s0126, s0244, s0046, s05...",[command_and_control],"[0.0067764707, -0.35248113, -0.62356573, 0.294...",0.901878
603,T1102.001,"[network_traffic_network_traffic_content, netw...","[network_traffic_content, network_traffic_flow]","[m1031, other]","[linux, windows, macos]","[s0373, s0128, s0674, s0531, s0455, s0013, s01...",[command_and_control],"[-0.7252811, -0.3269149, -0.49607247, 0.198911...",0.054010
604,T1102.003,"[network_traffic_network_connection_creation, ...","[network_connection_creation, network_traffic_...","[m1031, other]","[linux, windows, macos]","[s0455, other]",[command_and_control],"[-0.11939225, -0.35763258, -0.46435547, 0.5083...",-0.369923
605,T1047,"[command_command_execution, network_traffic_ne...","[command_execution, network_connection_creatio...","[m1040, m1038, m1026, m1018]",[windows],"[s0331, s0373, s0640, s0534, s1070, s1068, s00...",[execution],"[-0.63444924, -0.15962608, -0.2825894, 0.07326...",2.032367


In [127]:
pos_y = label_df[label_df['label'] == 1]
used_techniques=  pos_y['technique_ID'].unique()
all_techniques = technique_df['technique_ID'].unique()
unused_techniques = [t for t in all_techniques if t not in used_techniques]
print (unused_techniques)

['T1548', 'T1548.004', 'T1548.001', 'T1548.003', 'T1134.003', 'T1134.004', 'T1134.005', 'T1087', 'T1098.001', 'T1650', 'T1583', 'T1583.005', 'T1583.008', 'T1583.007', 'T1595', 'T1595.003', 'T1557.003', 'T1020.001', 'T1547', 'T1547.014', 'T1547.002', 'T1547.006', 'T1547.008', 'T1547.015', 'T1547.010', 'T1547.007', 'T1547.005', 'T1547.003', 'T1547.013', 'T1037.002', 'T1037.003', 'T1037.004', 'T1037.005', 'T1185', 'T1612', 'T1580', 'T1538', 'T1526', 'T1619', 'T1059.002', 'T1059.008', 'T1586', 'T1554', 'T1584', 'T1584.002', 'T1584.007', 'T1543', 'T1543.001', 'T1543.004', 'T1555.001', 'T1555.002', 'T1132', 'T1132.002', 'T1565', 'T1001', 'T1602', 'T1602.002', 'T1602.001', 'T1622', 'T1491', 'T1587.004', 'T1652', 'T1006', 'T1561', 'T1484', 'T1484.002', 'T1568.003', 'T1499.003', 'T1499.004', 'T1499.001', 'T1499.002', 'T1585.003', 'T1546', 'T1546.009', 'T1546.014', 'T1546.016', 'T1546.006', 'T1546.007', 'T1546.002', 'T1546.005', 'T1546.004', 'T1480', 'T1048.001', 'T1011', 'T1011.001', 'T1052', '

- Many unused techniques are siblings (share the same parent technique)
- ➡️ Initialize interaction rate for Unused technique by the statistics of the Used Technique of the same Tactic groups
- ❗Use technique used in TRAIN data only

In [128]:
t_interaction_rate = technique_df[[ 'input_technique_tactics', 'input_technique_interaction_rate']]
t_interaction_rate = t_interaction_rate.explode ('input_technique_tactics')
t_interaction_rate = t_interaction_rate.groupby (by= 'input_technique_tactics', as_index= False).agg ('mean')

unused_techniques = technique_df[technique_df['technique_ID'].isin(unused_techniques)]
unused_techniques = unused_techniques[['technique_ID', 'input_technique_tactics']]
unused_techniques = unused_techniques.explode ('input_technique_tactics')
unused_techniques = pd.merge (left = unused_techniques, right = t_interaction_rate, how = 'left', on = 'input_technique_tactics')
unused_techniques = unused_techniques[['technique_ID', 'input_technique_interaction_rate']]
unused_techniques = unused_techniques.groupby (by = 'technique_ID', as_index= False).agg ('mean')


In [129]:
for index, row in unused_techniques.iterrows():
    id_val = row['technique_ID']
    updated_val = row['input_technique_interaction_rate']
    
    # Locate the corresponding row in df_main and update the value
    technique_df.loc[technique_df['technique_ID'] == id_val, 'input_technique_interaction_rate'] = updated_val

In [130]:
technique_df

,technique_ID,input_technique_data_sources,input_technique_detection_name,input_technique_mitigation_id,input_technique_platforms,input_technique_software_id,input_technique_tactics,input_technique_description,input_technique_interaction_rate
0,T1548,"[command_command_execution, file_file_metadata...","[command_execution, file_metadata, file_modifi...","[m1047, m1038, m1028, m1026, m1022, other]","[linux, windows, macos]",[],"[defense_evasion, privilege_escalation]","[-0.36630338, -0.24036503, -0.36260876, 0.0984...",-0.174181
1,T1548.002,"[command_command_execution, process_process_cr...","[command_execution, process_creation, windows_...","[m1047, m1026, other]",[windows],"[s0584, s0640, s0606, s0570, s1068, s0089, s10...","[defense_evasion, privilege_escalation]","[-0.5305501, -0.43114784, -0.67361057, 0.16222...",0.477944
2,T1548.004,"[process_os_api_execution, process_process_cre...","[os_api_execution, process_creation]",[m1038],[macos],[s0402],"[defense_evasion, privilege_escalation]","[-0.57463825, -0.65824693, -0.23797776, -0.254...",-0.174181
3,T1548.001,"[command_command_execution, file_file_metadata...","[command_execution, file_metadata, file_modifi...",[m1028],"[linux, macos]",[other],"[defense_evasion, privilege_escalation]","[-0.7420097, -0.461457, -0.45979652, -0.116038...",-0.174181
4,T1548.003,"[command_command_execution, file_file_modifica...","[command_execution, file_modification, process...","[m1028, m1026, m1022]","[linux, macos]","[s0154, s0279, other]","[defense_evasion, privilege_escalation]","[-0.8037999, -0.6815256, 0.18332177, 0.0635958...",-0.174181
...,...,...,...,...,...,...,...,...,...
602,T1102.002,"[network_traffic_network_connection_creation, ...","[network_connection_creation, network_traffic_...","[m1031, other]","[linux, windows, macos]","[s0128, s0657, s0660, s0126, s0244, s0046, s05...",[command_and_control],"[0.0067764707, -0.35248113, -0.62356573, 0.294...",0.901878
603,T1102.001,"[network_traffic_network_traffic_content, netw...","[network_traffic_content, network_traffic_flow]","[m1031, other]","[linux, windows, macos]","[s0373, s0128, s0674, s0531, s0455, s0013, s01...",[command_and_control],"[-0.7252811, -0.3269149, -0.49607247, 0.198911...",0.054010
604,T1102.003,"[network_traffic_network_connection_creation, ...","[network_connection_creation, network_traffic_...","[m1031, other]","[linux, windows, macos]","[s0455, other]",[command_and_control],"[-0.11939225, -0.35763258, -0.46435547, 0.5083...",-0.369923
605,T1047,"[command_command_execution, network_traffic_ne...","[command_execution, network_connection_creatio...","[m1040, m1038, m1026, m1018]",[windows],"[s0331, s0373, s0640, s0534, s1070, s1068, s00...",[execution],"[-0.63444924, -0.15962608, -0.2825894, 0.07326...",2.032367
